In [1]:
import numpy as np
import h5py as h5
from pyproj import Proj,transform
from osgeo import gdal
from math import floor
import osr
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
class readGEDI():

    def __init__(self,fileList):
        '''Read GEDI data'''
        gedi_f = h5.File('/home/s1949330/Documents/initial_data/GEDI02_B_2020_04_30_2020124194427_O07882_01_T02809_02_003_01_V002.h5', 'r')
        beamList = ['BEAM0000','BEAM0001','BEAM0010','BEAM0011','BEAM0101','BEAM0110','BEAM1000','BEAM1011']           
        self.x = np.empty((), dtype = float)
        self.y = np.empty((), dtype = float)
        self.sens = np.empty((), dtype = float)
        self.cover = np.empty((), dtype = float)
        for beam in beamList:
            self.x = np.append(self.x, np.array(gedi_f[beam]['geolocation/lon_lowestmode']))
            self.y = np.append(self.y, np.array(gedi_f[beam]['geolocation/lat_lowestmode']))
            self.sens = np.append(self.sens, np.array(gedi_f[beam]['sensitivity']))
            self.cover = np.append(self.cover, np.array(gedi_f[beam]['cover'])) 
        return
            
    def reprojectGEDI (self,outEPSG):
        '''Reproject GEDI footprint coordinates'''
        inProj = Proj("epsg:4326")
        outProj = Proj("epsg:"+str(outEPSG))
        x, y = transform(inProj,outProj,gedi.y,gedi.x)
        gedi.x = x
        gedi.y = y
        return


    
class readSentinel():
    
    def __init__(self,fileName):
        '''Read Sentinel2 data'''
        ds = gdal.Open(fileName)
        proj = osr.SpatialReference(wkt = ds.GetProjection())
        self.epsg = int(proj.GetAttrValue('AUTHORITY', 1))
        self.nX = ds.RasterXSize
        self.nY = ds.RasterYSize
        transform_ds = ds.GetGeoTransform()
        self.xOrigin = transform_ds[0]
        self.yOrigin = transform_ds[3]
        self.pixelWidth = transform_ds[1]
        self.pixelHeight = transform_ds[5]
        self.data = ds.GetRasterBand(1).ReadAsArray(0, 0, self.nX, self.nY)
        return
    
    
    def coarsen(self,res):
        '''Coarsen resolution of non-20m Sentinel2 data'''    
        # Dimensions of output image
        newX = floor(self.nX * self.pixelWidth / res)
        newY = floor(self.nY * self.pixelHeight / (-1 * res))
        # Allocate space
        newData = np.zeros((newX,newY), dtype = float)
        contN = np.zeros((newX,newY), dtype = int)
        # Loop over input image and take rolling sum
        for i in range(0,self.nX):
            print('Column', i , 'of', self.nX)
            for j in range(0, self.nY):
                xInd = floor(i * self.pixelWidth / res)
                yInd = floor(j * -1 * self.pixelHeight / res)
                print(i,j,xInd,yInd,res,self.pixelWidth,self.pixelHeight)
                newData[xInd,yInd] += self.data[i,j]
                contN[xInd,yInd] += 1      
        # Normalise means
        newData[contN > 0] = newData[contN > 0] / contN[contN > 0]
        newData[contN == 0] = -999
        self.pixelWidth = res
        self.pixelHeight = -1 * res
        self.data = newData
        return
    
    
    def dumpGediSent(gedi,filename):
        '''Write GEDI and Sentinel2 (intersecting pixels) to csv'''
        f = open(filename,'w')
        for i in range(0,gedi.cover.shape[0]):
            if((gedi.cover[i] >= 0) & (gedi.s2out[i,0] >= 0)):
                line = str(gedi.cover[i]) + "," + str(gedi.x[i])+","+str(gedi.y[i]) + "," + str(gedi.sens[i])
                for j in range(0,gedi.s2out.shape[1]):
                    line = line + "," + str(gedi.s2out[i,j])
                line = line + "\n"
                f.write(line)
        f.close()
        print("Written to",filename)
        return

    

In [ ]:
#  Identify Sentinel2 pixels that intersect with GEDI footprints
xInd = np.array(np.floor((gedi.x - sent[0].xOrigin) / sent[0].pixelWidth), dtype = int)
yInd = np.array(np.floor((gedi.y - sent[0].yOrigin) / sent[0].pixelHeight), dtype = int)


# Prepare arrays to hold intersected Sentinel2 data
nGedi = gedi.cover.shape[0]
nSent = sent.shape[0]
gedi.s2out = np.full((nGedi, nSent), -999, dtype = float)


# Loop over GEDI footprints
for j in range(0,gedi.cover.shape[0]):
    if((xInd[j]>=0)&(xInd[j]<sent[0].nX)&(yInd[j]>=0)&(yInd[j]<sent[0].nY)):
        # Loop over Sentinel2 bands
        for i in range(0,nSent):
            gedi.s2out[j,i]=sent[i].data[xInd[j],yInd[j]]

            
# Write csv for GEDI (reprojected x and y, sensitivity, cover) + Sentinel2 (intersecting pixels for 12 20m bands)   
dumpGediSent('pilot.csv')


In [ ]:
# Calculate Sentinel2 NDVI

band8 = sent[7]
band4 = sent[3]
ndvi = (band8.astype(float) - band4.astype(float)) / (band8.astype(float) + band4.astype(float))


plt.imshow(ndvi, cmap="RdYlGn")
plt.colorbar()
plt.show()
# Plot NDVI against GEDI cover

plt.imshow(ndvi, cmap = 'Greens')

